<a href="https://colab.research.google.com/github/hariPrasadCoder/10-pointer-analysis/blob/main/Preparing_the_dataset_with_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install yfinance
import yfinance as yf
import numpy as np
from sklearn import preprocessing

In [ ]:
df = pd.read_csv('scraped_data.csv')

In [ ]:
price_trend = []
price_score = []
for i in df['ticker']:
  try:
    stock = yf.Ticker(i)
    df1 = stock.history(period='1y')
    df1.reset_index(inplace=True)

    df1['year'] = pd.DatetimeIndex(df1['Date']).year
    df1['month'] = pd.DatetimeIndex(df1['Date']).month

    df1 = df1[df1.year != 2022]
    df1 = df1[['month', 'Close']]
    df1 = df1.groupby(['month']).median()


    def trendline(index,data, order=1):
        coeffs = np.polyfit(index, list(data), order)
        slope = coeffs[-2]
        return float(slope)

    index=[1,2,3,4,5,6,7,8,9,10,11,12]
    resultant=trendline(index,df1['Close'])
    price_trend.append(resultant)

    df1.reset_index(inplace=True)
    initial = df1['Close'][df1['month'] == 1].values[0]
    final = df1['Close'][df1['month'] == 12].values[0]

    price_change = ((final - initial)/initial)*100
    price_score.append(price_change)

  except:
    price_trend.append('NA')
    price_score.append('NA')

df['price_trend'] = price_trend
df['price_score'] = price_score

- BRK.B: No data found, symbol may be delisted


In [ ]:
df = df[df.price_trend != 'NA']
df = df[df.price_score != 'NA']

In [ ]:
x = df['price_trend'].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x = x.reshape(-1,1)
x_scaled = min_max_scaler.fit_transform(x)
df['price_trend'] = x_scaled * 100
df

Unnamed: 0 ticker  ... price_trend  price_score
0             0    ABT  ...   13.154162      22.1353
1             1   ABBV  ...   13.362954      23.6857
2             2   ABMD  ...   13.998170      -4.0271
3             3    ACN  ...   22.183968      50.5299
4             4    AAP  ...   17.394116      43.1487
..          ...    ...  ...         ...          ...
317         317   XLNX  ...   18.107216      52.6718
318         318    XYL  ...   14.393375      16.1458
319         319    YUM  ...   14.191796      26.3353
320         320    ZBH  ...    9.304188     -21.2518
321         321   ZION  ...   13.095174      31.8065

[321 rows x 15 columns]

In [ ]:
df.to_csv('final_data.csv', index=False)
